<a href="https://colab.research.google.com/github/dr-code/colab/blob/main/flightScreenshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install required dependencies
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install selenium pillow pydrive gspread oauth2client

# Import required libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from PIL import Image
import time
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Authenticate Google Drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

# Get flight URL from Google Drive file
file_list = drive.ListFile({'q': "title = 'flight_details.txt'"}).GetList()
if len(file_list) > 0:
    file = file_list[0]
    flight_url = file.GetContentString()
    print("Using flight URL from Google Drive:", flight_url)
else:
    print("Flight details file not found. Exiting...")
    exit()

# Set up Selenium WebDriver for Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
service = Service("/usr/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

# Open the FlightStats page with the updated URL
driver.get(flight_url)

# Wait for the page to load
time.sleep(5)

# Locate the flight details section
element = driver.find_element(By.CSS_SELECTOR, "section .historical-flight-header")

# Capture full-page screenshot
driver.save_screenshot("full_screenshot.png")

# Crop only the required section
location = element.location
size = element.size
image = Image.open("full_screenshot.png")
left, top = location['x'], location['y']
right, bottom = left + size['width'], top + size['height']
cropped_image = image.crop((left, top, right, bottom))
cropped_image.save("cropped_screenshot.png")

# Upload to Google Drive
screenshot_file = drive.CreateFile({'title': 'FlightStats_Screenshot.png', 'mimeType': 'image/png'})
screenshot_file.SetContentFile("cropped_screenshot.png")
screenshot_file.Upload()

# Get the public link of the file
screenshot_file.InsertPermission({'type': 'anyone', 'value': 'anyone', 'role': 'reader'})
file_url = screenshot_file['webContentLink']

# Save the public link to Google Sheets
sheet_id = "1gzK5fPqH9OW-AfKklK7aUeMnjdwKKHHfINq6Lz2ZMUU"
sheet_name = "Flights"

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/Colab Notebooks/chatgpt-446517-f9f9353e526c.json", scope)
client = gspread.authorize(creds)

sheet = client.open_by_key(sheet_id).worksheet(sheet_name)
sheet.update_acell("Q1", file_url)

# Close the browser
driver.quit()

print("Screenshot saved to Google Drive:", file_url)


ModuleNotFoundError: No module named 'selenium'